In [8]:
import pyranges as pr
import pandas as pd
import seaborn as sns
import numpy as np
import sqlite3
import gffutils
import time
import pybedtools

In [2]:
start_time = time.time()
human_gr = pr.read_gtf("/home/U210050044/data/DH607-project/Homo_sapiens.GRCh38.112.chr.gtf")
end_time = time.time()
print(f"Finished reading into pyranges object in time {end_time-start_time}")

Finished reading into pyranges object in time 64.55301427841187


In [3]:
start_time = time.time()
human_db = gffutils.create_db("/home/U210050044/data/DH607-project/Homo_sapiens.GRCh38.112.chr.gtf","/home/U210050044/data/DH607-project/human_db.sqlite3",disable_infer_genes=True,disable_infer_transcripts=True, force=True)
end_time = time.time()
print(f"Finished converting gtf to sql db in time {end_time-start_time}")

Finished converting gtf to sql db in time 449.48835945129395


In [3]:
# Open the database
human_db = gffutils.FeatureDB("/home/U210050044/data/DH607-project/human_db.sqlite3")

In [4]:
conn = sqlite3.connect(":memory:")
start_time = time.time()
human_gr.df.to_sql("human", conn, if_exists="replace")
end_time = time.time()
print(f"Converted Pyranges object to in-memory sql db in time {end_time-start_time}")
cur = conn.cursor()

Converted Pyranges object to in-memory sql db in time 49.05035901069641


In [6]:
# Aggregate Queries: Counting transcripts, calculating exon lengths, and grouping features
### Aggregate Query 1: Count the number of exons for each gene
# Using pyranges
%timeit exon_counts_pr = (human_gr[human_gr.Feature == "exon"].df.groupby("gene_id").size().reset_index(name="exon_count"))

5.12 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# Using gffutils
def exon_counts_gffutls():
    # Iterate over all genes and count their exons
    exon_counts = {}
    for gene in human_db.features_of_type("gene"):
        # Efficiently count exons for each gene using the `children` method
        exon_counts[gene.id] = sum(
            1 for _ in human_db.children(gene, featuretype="exon")
    )
    return exon_counts
%timeit exon_counts_gff = exon_counts_gffutls()

44 s ± 32.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# Using sql
%timeit exon_counts_sql = pd.read_sql_query("SELECT gene_id, COUNT(*) as exon_count FROM human WHERE Feature = 'exon' GROUP BY gene_id", conn)

1.4 s ± 21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
### Aggregate Query 2: Calculate the total length of exons for each gene
# Using pyranges
%timeit exon_lengths_pr = (human_gr[human_gr.Feature == "exon"].df.assign(length=lambda df: df["End"] - df["Start"]).groupby("gene_id")["length"].sum().reset_index(name="total_exon_length"))

5.77 s ± 56.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# Using gffutils
def exon_lengths_gffutls():
    # Dictionary to store total exon length per gene
    exon_lengths = {}
    # Iterate over all genes and calculate total exon length
    for gene in human_db.features_of_type("gene"):
        # Compute total exon length for each gene
        exon_lengths[gene.id] = sum(
            (child.end - child.start + 1)
            for child in human_db.children(gene, featuretype="exon")
        )
    return exon_lengths
%timeit exon_lengths_gff = exon_lengths_gffutls()

43.8 s ± 80.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# using sql
%timeit exon_lengths_sql = pd.read_sql_query("SELECT gene_id, SUM(End - Start) as total_exon_length FROM human WHERE Feature = 'exon' GROUP BY gene_id", conn)

1.59 s ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
### Aggregate Query 3: Identify the chromosome with the highest number of transcripts
# Using pyranges
%timeit transcript_counts_pr = human_gr[human_gr.Feature == "transcript"].df["Chromosome"].value_counts().idxmax()

953 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
# Using gffutils
def transcript_counts_gffutls():
    # Dictionary to store the number of transcripts per chromosome
    transcript_counts = {}
    # Iterate over all transcripts and count the number of transcripts per chromosome
    for transcript in human_db.features_of_type("transcript"):
        chrom = transcript.chrom
        if chrom not in transcript_counts:
            transcript_counts[chrom] = 0
        transcript_counts[chrom] += 1
    return max(transcript_counts, key=transcript_counts.get)
%timeit transcript_counts_gff = transcript_counts_gffutls()

5.52 s ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# using sql
%timeit transcript_counts_sql = pd.read_sql_query("SELECT Chromosome, COUNT(*) as transcript_count FROM human WHERE Feature = 'transcript' GROUP BY Chromosome ORDER BY transcript_count DESC LIMIT 1", conn)

531 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Interval Arithmetic

In [ ]:
# Query 1: Merging Overlapping Exon Intervals
# Combine all overlapping exon intervals into contiguous regions
# Using pyranges
%timeit exon_intervals_pr = human_gr[human_gr.Feature == "exon"].merge(strand=False)

1.76 s ± 57.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# using gffutils TODO: run
def exon_intervals_gffutls():
    exon_intervals = [
        gffutils.helpers.asinterval(exon) for exon in human_db.features_of_type("exon")
    ]
    exon_intervals_sorted = sorted(exon_intervals, key=lambda x: (x.chrom, x.start))
    # Use pybedtools to merge overlapping intervals
    return pybedtools.BedTool(exon_intervals_sorted).merge()
%timeit exon_intervals_gff = exon_intervals_gffutls()

In [93]:
# using sql
def find_exon_intervals_sql():
    exon_intervals = pd.read_sql_query("SELECT Chromosome, Start, End FROM human WHERE Feature = 'exon' ORDER BY Chromosome, Start", conn)
    return pybedtools.BedTool.from_dataframe(exon_intervals).merge()
%timeit exon_intervals_sql = find_exon_intervals_sql()

6.63 s ± 21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Query 2: Finding Overlaps with a Specific Interval
# Find all gene features that overlap a given interval chr1:100000-200000
# Using pyranges
%timeit overlapping_genes_pr = human_gr[human_gr.Feature == "gene"].overlap(pr.from_dict({"Chromosome": ["1"], "Start": [100000], "End": [200000]}), strand=False)

399 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [144]:
# using gffutils TODO: run
def overlapping_genes_gffutls():
    exon_intervals_overlapping = pybedtools.BedTool.from_dataframe(pd.DataFrame({"Chromosome": ["1"], "Start": [100000], "End": [200000]}))
    exon_intervals = [
        gffutils.helpers.asinterval(exon) for exon in human_db.features_of_type("exon")
    ]
    exon_intervals_sorted = sorted(exon_intervals, key=lambda x: (x.chrom, x.start))
    # Use pybedtools to merge overlapping intervals
    return exon_intervals_overlapping.window(pybedtools.BedTool(exon_intervals_sorted).merge(), w=0)
%timeit overlapping_genes_gff = overlapping_genes_gffutls()

1min 29s ± 414 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [117]:
# using sql
def find_overlapping_genes_sql():
    exon_intervals_overlapping = pybedtools.BedTool.from_dataframe(pd.DataFrame({"Chromosome": ["1"], "Start": [100000], "End": [200000]}))
    exon_intervals = pd.read_sql_query("SELECT Chromosome, Start, End FROM human WHERE Feature = 'exon' ORDER BY Chromosome, Start", conn)
    return exon_intervals_overlapping.window(pybedtools.BedTool.from_dataframe(exon_intervals).merge(), w=0)
%timeit overlapping_genes_sql = find_overlapping_genes_sql()

7.41 s ± 51.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [143]:
# Query 3: Subtracting Intervals
# Remove a set of repetitive regions from the exon features.
# Using pyranges
%timeit subtracted_intervals_pr = human_gr[human_gr.Feature == "exon"].merge(strand=False).subtract(pr.from_dict({"Chromosome": ["1", "1"],"Start": [15000000, 20000000],"End": [16000000, 21000000],}))

4.38 s ± 74.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [145]:
# using gffutils TODO: run
def subtracted_intervals_gffutls():
    exon_intervals_subtract = pybedtools.BedTool.from_dataframe(pd.DataFrame({"Chromosome": ["1", "1"], "Start": [15000000, 20000000], "End": [16000000, 21000000]}))
    exon_intervals = [
        gffutils.helpers.asinterval(exon) for exon in human_db.features_of_type("exon")
    ]
    exon_intervals_sorted = sorted(exon_intervals, key=lambda x: (x.chrom, x.start))
    # Use pybedtools to merge overlapping intervals
    return pybedtools.BedTool(exon_intervals_sorted).merge().subtract(exon_intervals_subtract)
%timeit overlapping_genes_gff = overlapping_genes_gffutls()

1min 28s ± 441 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [142]:
# using sql
def find_subtracted_intervals_sql():
    exon_intervals_subtract = pybedtools.BedTool.from_dataframe(pd.DataFrame({"Chromosome": ["1", "1"], "Start": [15000000, 20000000], "End": [16000000, 21000000]}))
    exon_intervals = pd.read_sql_query("SELECT Chromosome, Start, End FROM human WHERE Feature = 'exon' ORDER BY Chromosome, Start", conn)
    # Use pybedtools to merge overlapping intervals
    return pybedtools.BedTool.from_dataframe(exon_intervals).merge().subtract(exon_intervals_subtract)
%timeit subtracted_intervals_sql = find_subtracted_intervals_sql()

7.28 s ± 37.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
